In [2]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import linalg
import math

In [104]:
def T(i: int, j: int, A: np.ndarray):
    x = -2 * A[i,j]
    y = A[i,i] - A[j,j]

    if y == 0:
        cos = 1 / np.sqrt(2)
        sin = cos
    else:
        cos = np.sqrt(1/2*(1 + np.abs(y)/ np.sqrt(x**2 + y**2)))
        sin = np.sign(x*y) * np.abs(x) / (2*cos*np.sqrt(x**2 + y**2))

    res = np.eye(A.shape[0])
    res[i, i] = cos
    res[i, j] = -sin
    res[j, i] = sin
    res[j, j] = cos

    return res

In [135]:
def max_elem(A: np.ndarray):
    B = np.abs(A)
    np.fill_diagonal(B, -1)
    return np.unravel_index(np.argmax(B), B.shape)

def cycle_choise(A: np.ndarray, k: int):
    i, j = np.tril_indices_from(A, k = -1)
    idx = (k-1) % len(i)
    return (i[idx], j[idx])

def err(A: np.ndarray):
    mask = np.invert(np.eye(A.shape[0], dtype=bool))
    return np.sum(A*A, where=mask, dtype=np.float64)

def R_i(i, A):
    mask = np.ones(A.shape[0], dtype=bool)
    mask[i] = False
    return np.sum(np.abs(A[i, mask]), dtype=np.float64)

def borders(A):
    min_ = 1000000000
    max_ = -1000000000
    for i in range(A.shape[0]):
        min_ = min(min_, A[i, i] - R_i(i, A))
        max_ = max(max_, A[i, i] + R_i(i, A))
    return min_, max_

def create_b(A):
    mask = np.eye(A.shape[0], dtype=bool)
    B = A.copy()
    B[mask] = 0
    b = np.sum(B*B, axis=1)
    return b

def update_b(i, j, b, A):
    mask = np.ones(A.shape[0], dtype=bool)
    mask[i] = False
    b[i] = np.sum((A*A)[i,mask], dtype=np.float64)
    mask[i] = True
    mask[j] = False
    b[j] = np.sum((A*A)[j,mask], dtype=np.float64)
    return b

def opt_choise(A, b):
    i = np.argmax(np.abs(b))
    q = np.abs(A[i], dtype=np.float64)
    q[i] = -1
    j = np.argmax(q)
    return i, j

In [149]:
#A = linalg.pascal(5)
A = linalg.hilbert(5)
#A = np.array([[1, -2, 2],[-2, -2, 4],[2,4,-2]])
eps = 1e-15

A_prev = A.copy()
A_new = A_prev.copy()
#b_ = create_b(A)

k = 1
while True:
    #i, j = cycle_choise(A_prev, k)
    i, j = max_elem(A_prev)
    #i, j = opt_choise(A_prev, b_)
    #b_ = update_b(i, j, b_, A)
    T_curr = T(i, j, A_prev)
    A_new = T_curr @ A_prev @ T_curr.T

    if (err(A_new) < eps):
        break
    
    A_prev = A_new
    k += 1

print("Number of iterations: ", k)
eigs = np.sort(np.diag(A_new))
orig_eigs = np.sort(linalg.eigh(A)[0])

print("Eigenvalues: ", list(eigs))
print("Relative error: ", linalg.norm(orig_eigs-eigs) / linalg.norm(orig_eigs))
print("Lower and upper limit for eigvals: ", borders(A))



Number of iterations:  29
Eigenvalues:  [3.287928772179718e-06, 0.00030589804015119886, 0.01140749162341981, 0.20853421861101326, 1.5670506910982307]
Relative error:  1.132837091885977e-15
Lower and upper limit for eigvals:  (-0.7833333333333334, 2.283333333333333)
